In [330]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
np.seterr(divide = 'ignore') 

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [193]:
df = pd.read_csv('Covid Live.csv', index_col='#')
df.columns = list(map(lambda x: x.replace('\n', ' '), df.columns))
for col in df.columns.drop(['Country, Other', 'New Deaths']):
    df[col] = df[col].str.replace(',', '')
    df[col] = df[col].astype('float')

In [194]:
countries = pd.read_csv('countries of the world.csv', decimal=',')
countries['Country'] = countries['Country'].str.strip().str.replace('&', 'and')
countries.shape

(227, 20)

In [195]:
import pycountry_convert as pc

In [196]:
def country_to_continent(country_name):
    if country_name in ['Sint Maarten', 'St. Vincent Grenadines']:
        return 'North America'
    if country_name == 'Vatican City':
        return 'Europe'
    if country_name == 'Western Sahara':
        return 'Africa'
    country_alpha2 = pc.country_name_to_country_alpha2(country_name)
    if country_alpha2 == 'TL':
        return 'Asia'
    else:
        country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)        
        country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
    return country_continent_name

In [197]:
countries = countries.replace({
    'Bahamas, The': 'Bahamas', 
    'Virgin Islands': 'Virgin Islands, U.S.', 
    'Burma': 'Myanmar', 
    'Central African Rep.': 'Central African Republic',
    'Congo, Dem. Rep.': 'Congo', 
    "Cote d'Ivoire": "Côte d'Ivoire",
    'Gambia, The': 'Gambia',
    'Korea, North': 'North Korea',
    'Korea, South': 'South Korea', 
    'Micronesia, Fed. St.': 'Micronesia', 
    'N. Mariana Islands':'Northern Mariana Islands', 
    'Reunion':'Réunion',
    'Saint Helena': 'Saint Helena, Ascension and Tristan da Cunha',
    'St Pierre and Miquelon':'Saint Pierre and Miquelon', 
    'Turks and Caicos Is':'Turks and Caicos Islands'
})

df = df.replace({
    'S. Korea': 'South Korea',
    'UK': 'United Kingdom',
    'DPRK': 'North Korea',
    'UAE': 'United Arab Emirates',
    'DRC': 'Congo',
    'Faeroe Islands': 'Faroe Islands',
    'CAR': 'Central African Republic', 
    'Caribbean Netherlands': 'Bonaire', 
    'St. Barth': 'Saint Barthélemy',
    'Saint Pierre Miquelon': 'Saint Pierre and Miquelon',
    'Saint Helena': 'Saint Helena, Ascension and Tristan da Cunha'
})

df = df[~df['Country, Other'].isin(['Channel Islands', 'Diamond Princess', 'MS Zaandam', 
                                    'Sint Maarten', 'St. Vincent Grenadines', 'Vatican City'])]
countries = countries[~countries['Country'].isin(['Congo, Repub. of the', 'Gaza Strip', 
                                                  'Netherlands Antilles', 'British Virgin Is.',
                                                  'West Bank'])]

In [198]:
df['Alpha3 code'] = list(map(pc.country_name_to_country_alpha3, df['Country, Other']))
countries['Alpha3 code'] = list(map(pc.country_name_to_country_alpha3, countries['Country']))
countries = countries.drop('Population', axis=1)
df = pd.merge(df, countries, on='Alpha3 code')
df['Continent'] = df['Country, Other'].apply(country_to_continent)
df = df.drop(['Country'], axis=1)

In [199]:
import plotly.graph_objects as go
from ipywidgets import widgets
import plotly.io as pio
pio.renderers.default = "iframe"

In [202]:
def show_fig(df):
    fig = px.histogram(df, x='Deaths/ 1M pop', color='Continent', nbins=10, barmode='group', title='Deaths/ 1M pop')
    f = go.FigureWidget(fig)
    options = df.dtypes[df.dtypes != 'object'].index.to_list()
    x_widget = widgets.Dropdown(options=options, 
                                value='Deaths/ 1M pop', 
                                description='X axis:')
    continents = df['Continent'].unique()
    
    lst_bins = np.arange(5, 51, 5)
    n_bins_widget = widgets.Dropdown(options=lst_bins, 
                                     value=10,
                                     description='Number of bins:')
    
    barmode_lst = ['group', 'overlay', 'stack']
    barmode_widget = widgets.Dropdown(options=barmode_lst, 
                                      value='group', 
                                      description='Barmode:')
    
    barnorm_lst = ['fraction', 'percent', 'default']
    barnorm_widget = widgets.Dropdown(options=barnorm_lst,
                                      value='default',
                                      description='Barnorm:')
    
    def validate():
        return x_widget.value in options and n_bins_widget.value in lst_bins \
            and barmode_widget.value in barmode_lst and barnorm_widget.value in barnorm_lst
    
    def response(change):
        if validate():
            col = x_widget.value
            nbins = n_bins_widget.value
            barmode_val = barmode_widget.value
            barnorm_val = barnorm_widget.value if barnorm_widget.value != 'default' else None
            for i, c in enumerate(continents):
                _temp = df[df['Continent'] == c][col]
                f.data[i].x = _temp.values
                f.data[i].nbinsx = int(nbins)
                f.data[i].marker.opacity = 0.5 if barmode_val == 'overlay' else 1
                f.layout.title.text = col
                f.layout.barnorm = barnorm_val
                f.layout.barmode = barmode_val
                
    x_widget.observe(response, names='value')
    n_bins_widget.observe(response, names='value')
    barmode_widget.observe(response, names='value')
    barnorm_widget.observe(response, names='value')
    
    container = widgets.HBox([x_widget, n_bins_widget, barmode_widget, barnorm_widget])
    box = widgets.VBox([container, f])
    return box

In [204]:
fig = show_fig(df)
fig

In [205]:
def show_scatter(df):
    fig = px.scatter(df, x='Deaths/ 1M pop', y='Deathrate', color='Continent', title='Scatter plot', hover_name='Country, Other', width=600, height=600)
    
    f = go.FigureWidget(fig)
    options = df.dtypes[df.dtypes != 'object'].index.to_list()
    x_widget = widgets.Dropdown(options=options, 
                                value='Deaths/ 1M pop', 
                                description='X axis:')
    
    y_widget = widgets.Dropdown(options=options, 
                                value='Deathrate', 
                                description='Y axis:')
    
    continents = df['Continent'].unique()
    
    def validate():
        return x_widget.value in options and y_widget.value in options
    
    def response(change):
        if validate():
            x_val = x_widget.value
            y_val = y_widget.value
            for i, c in enumerate(continents):
                _temp = df[df['Continent'] == c]
                f.data[i].x = _temp[x_val].values
                f.data[i].y = _temp[y_val].values
                f.layout.xaxis.title.text = x_val
                f.layout.yaxis.title.text = y_val
                
    x_widget.observe(response, names='value')
    y_widget.observe(response, names='value')
    
    container = widgets.HBox([x_widget, y_widget])
    box = widgets.VBox([container, f])
    return box

In [206]:
show_scatter(df)

In [221]:
def show_heatmap(df):
    heatmap = px.imshow(df[df.dtypes[df.dtypes != 'object'].index].corr(), color_continuous_midpoint=0,
                        color_continuous_scale='tropic', height=800, width=800, title = 'Heatmap (All continents)')
    f = go.FigureWidget(heatmap)
    
    continents = list(df['Continent'].unique()) + ['All']
    continent_widget = widgets.Dropdown(options=continents, 
                                        value='All', 
                                        description='Continent:')
    
    def validate():
        return continent_widget.value in continents
    
    def response(change):
        if validate():
            c_val = continent_widget.value
            if c_val == 'All':
                f.data[0].z = df[df.dtypes[df.dtypes != 'object'].index].corr()
                f.layout.title.text = 'Heatmap (all continents)'
            else:
                _temp = df[df['Continent'] == c_val]
                f.data[0].z = _temp[_temp.dtypes[_temp.dtypes != 'object'].index].corr()
                f.layout.title.text = f'Heatmap ({c_val})'
                
    continent_widget.observe(response, names='value')
    return widgets.VBox([continent_widget, f])

In [159]:
fig = px.bar(df, x='Continent', y='Climate', color='Crops (%)', 
             hover_name='Country, Other', text='Country, Other',
             title='Bar plot', color_continuous_scale='tropic')
fig.update_traces(textposition='inside')

In [308]:
df['Alpha3 code'] = list(map(pc.country_name_to_country_alpha3, df['Country, Other']))

In [353]:
s = 'Asia'
s.lower()

'asia'

In [400]:
df['Region'] = df['Region'].str.strip().str.title()

In [368]:
def draw_map(df):

    fig = px.choropleth(df, 
                        locations='Alpha3 code', 
                        color='Population', 
                        featureidkey='properties.Population', scope='world', hover_name='Country, Other', 
                        hover_data={'Alpha3 code':False}, title='Population around the world', 
                        color_continuous_scale='tropic', projection='equirectangular',
                        )
    
    f = go.FigureWidget(fig)
    
    options = df.dtypes[df.dtypes != 'object'].index
    col_widget = widgets.Dropdown(options=options, 
                                  value='Population', 
                                  description='Feature:')
    
    scopes = ['World', 'North America', 'Asia', 'Europe', 'South America', 'Africa']
    scope_widget = widgets.Dropdown(options=scopes, 
                                    value='World',
                                    description='Continent:')
    
    scales = ['default', 'log']
    scale_widget = widgets.Dropdown(options=scales,
                                    value='default',
                                    description='Scale:')
    
    all_projections = ['equirectangular', 'mercator', 'orthographic', 'natural earth', 
                       'kavrayskiy7', 'miller', 'robinson', 'eckert4', 'azimuthal equal area', 
                       'azimuthal equidistant', 'conic equal area', 'conic conformal', 
                       'conic equidistant', 'gnomonic', 'stereographic', 'mollweide', 'hammer', 
                       'transverse mercator', 'winkel tripel', 'aitoff', 'sinusoidal']
    projection_widget = widgets.Dropdown(options=all_projections, 
                                         value='equirectangular',
                                         description='Projection:')
    
    
    def validate():
        return col_widget.value in options and scope_widget.value in scopes and \
               scale_widget.value in scales and projection_widget.value in all_projections
    
    def response(change):
        if validate():
            col = col_widget.value
            scope_val = scope_widget.value.lower()
            scale_val = scale_widget.value
            projection = projection_widget.value
            
            f.data[0].featureidkey = f'properties.{col}'
            if scale_val == 'log':
                f.data[0].z = np.log(df[col].values)
            else:
                f.data[0].z = df[col].values
            f.layout.coloraxis.colorbar.title.text = col
            f.layout.geo.scope = scope_val
            if scope_val == 'world':
                f.layout.title.text = f'{col} around the world'
            else:
                f.layout.title.text = f'{col} in {scope_val.title()}'
            if scale_val == 'log':
                f.layout.title.text = f.layout.title.text + ' (logarithmic scale)'
            f.layout.geo.projection.type = projection
                
    col_widget.observe(response, names='value')
    scope_widget.observe(response, names='value')
    scale_widget.observe(response, names='value')
    projection_widget.observe(response, names='value')
    
    container = widgets.HBox([col_widget, scope_widget, scale_widget, projection_widget])
    return widgets.VBox([container, f])

In [370]:
# draw_map(df)

In [453]:
def plot_sunburst(df):
    fig = px.sunburst(df,
                      path=['Continent', 'Region', 'Country, Other'],
                      values="Total Cases",
                      title="Sunburst plot (feature: Total Cases)", branchvalues='total', height=500, width=500)
    f = go.FigureWidget(fig)
    
    options = df.dtypes[df.dtypes != 'object'].index
    
    col_widget = widgets.Dropdown(options=options,
                                  value='Total Cases', 
                                  description='Feature:')
    lst = f.data[0].ids
    
    def create_values_array(column, df=df, lst=lst):
        vals = []
        for l in lst:
            kek = l.split('/')
            if len(kek) == 3:
                _temp = df.groupby(by='Country, Other')[column]
                vals.append(_temp.sum().loc[kek[-1]])
            elif len(kek) == 2:
                _temp = df.groupby(by=['Continent', 'Region'])[column]
                vals.append(_temp.sum().loc[(kek[-2], kek[-1])])
            else:
                _temp = df.groupby(by='Continent')[column]
                vals.append(_temp.sum().loc[kek[0]])
        return vals
    
    def validate():
        return col_widget.value in options
    
    def response(change):
        if validate():
            col = col_widget.value
            f.data[0].values = create_values_array(col)
            f.layout.title.text = f"Sunburst plot (feature: {col})"
            
    col_widget.observe(response, names='value')
    return widgets.VBox([col_widget, f])

In [455]:
# plot_sunburst(df)